In [2]:
import numpy as np
import pandas as pd
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

## Data

In [3]:
acs = pd.read_csv('../modeling/final_acs_transportation_choice.csv')#, index_col=0)
acs.head()

,taxi_zone,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5),P(mode6)
0,3.0,0.115434,17.843262,7.334361,3885.402712,478.622467,4316.681764
1,4.0,42.851015,140.891180,84.609811,6937.780033,2631.714648,779.153313
2,5.0,0.081377,13.158607,2.120444,1860.706347,40.495673,6312.437553
3,6.0,0.109017,7.637848,1.878344,1974.772111,198.118892,3086.483787
4,7.0,25.522468,142.614028,37.582529,28436.765508,2680.397880,5365.117587


In [4]:
mode_data = pd.read_csv('../modeling/final_allMode_with_2017wage_cleaned_update.csv', index_col=0)
mode_data = mode_data.dropna()
mode_data.head()

,DOlocationID,ODpair,PUlocationID,duration,mode,nest,price,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000
0,4,3-4,3,39.695000,2,1,64.000,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005
1,4,3-4,3,45.216667,3,1,61.500,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005
2,4,3-4,3,83.000000,4,2,5.500,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005
3,4,3-4,3,225.933333,5,3,0.000,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005
4,4,3-4,3,39.695000,6,4,29.424,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005


### Apply to our case

In [5]:
def compare_with_ground_truth(predictdf, truedf):
    '''
    compare our predicted transportation choice with ground truth
    
    The header of the datafrme after merge (named 'data_compare') should be like:
    taxi_zone | P(mode1)_x | P(mode2)_x | P(mode3)_x | P(mode4)_x | P(mode5)_x | P(mode1)_y | P(mode2)_y | P(mode3)_y | P(mode4)_y | P(mode5)_y
    '''
    import numpy as np
    # makesure predictdf and truedf have the same formats
    for col in predictdf.columns:
        predictdf[col] = predictdf[col].astype(float) 
    predictdf = predictdf.fillna(0)
    predictdf = predictdf.replace([np.inf, -np.inf], np.nan)
    predictdf = predictdf.dropna()
    predictdf['taxi_zone'] = predictdf.index.map(lambda x: x.split('-')[0]) #get origin taxi zone from each OD pair
    predictdf = predictdf.groupby('taxi_zone').sum().reset_index() #group the popuation by taxi zone
    predictdf['taxi_zone'] = predictdf['taxi_zone'].astype(int)
    
    #do same thing as predictdf to loss_function_deno dataframe
#     for col in loss_function_deno.columns:
#         loss_function_deno[col] = loss_function_deno[col].astype(float) 
#     loss_function_deno = loss_function_deno.fillna(0)
#     loss_function_deno = loss_function_deno.replace([np.inf, -np.inf], np.nan)
#     loss_function_deno = loss_function_deno.dropna()
#     loss_function_deno['taxi_zone'] = loss_function_deno.index.map(lambda x: x.split('-')[0]) #get origin taxi zone from each OD pair
#     loss_function_deno = loss_function_deno.groupby('taxi_zone').sum().reset_index() #group the popuation by taxi zone
#     loss_function_deno['taxi_zone'] = loss_function_deno['taxi_zone'].astype(int)   
    
    truedf['taxi_zone'] = truedf['taxi_zone'].astype(int)
    
    data_compare = pd.merge(predictdf, truedf, left_on='taxi_zone', right_on = 'taxi_zone')

    data_compare = data_compare.dropna() 
    data_compare_zone = data_compare['taxi_zone'].unique()
#     loss_function_deno = loss_function_deno[loss_function_deno['taxi_zone'].isin(data_compare_zone)]

    rloss = 0
    rmse = 0
    for i in range(1,totmode+1):
        data_compare_deno = (data_compare[data_compare.columns[i+totmode]].replace(to_replace=0,value = 1)).copy()
        rlossi = (np.sqrt(sum(((data_compare[data_compare.columns[i]] - data_compare[data_compare.columns[i+totmode]])**2)\
                        /(data_compare_deno))))
        rmsei = (np.sqrt(sum((data_compare[data_compare.columns[i]] - data_compare[data_compare.columns[i+totmode]])**2)))
        if rlossi == np.nan: print('Nan encountered')
    rloss += rlossi
    rmse += rmsei
    tot_mse = np.sqrt(sum((np.asarray(predictdf.sum()[1:]) - np.asarray(truedf.sum()[1:]))**2))
    return rloss, rmse, tot_mse

In [6]:
totmode = 6

1

In [7]:
predict_choice_1 = pd.read_csv('Final_results/results_scenario1_.04_1.5_for_uncertainty.csv',\
                               index_col=0)

In [8]:
rloss1, rmse1, tot_mse1 = compare_with_ground_truth(predict_choice_1, acs)

In [9]:
predict_choice_1.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5),P(mode6)
3-4,0.0,0.0,0.0,3.877147,0.249570,1.873283
3-7,0.0,0.0,0.0,12.012387,1.458571,22.529042
3-9,0.0,0.0,0.0,0.140546,0.072559,1.786895
3-10,0.0,0.0,0.0,0.157064,0.124834,2.718103
3-11,0.0,0.0,0.0,3.853464,0.145975,0.000561


2

In [10]:
predict_choice_2 = pd.read_csv('Final_results/results_scenario1_0.05_1.5.csv',index_col=0)

In [11]:
rloss2, rmse2, tot_mse2 = compare_with_ground_truth(predict_choice_2, acs)

3

In [12]:
predict_choice_3 = pd.read_csv('Final_results/results_scenario1_.06_1.5.csv',index_col=0)

In [13]:
rloss3, rmse3, tot_mse3 = compare_with_ground_truth(predict_choice_3, acs)

4

In [14]:
predict_choice_4 = pd.read_csv('Final_results/results_scenario1_.05_1.4_for_uncertainty.csv',index_col=0)

In [15]:
rloss4, rmse4, tot_mse4 = compare_with_ground_truth(predict_choice_4, acs)

5

In [16]:
predict_choice_5 = pd.read_csv('Final_results/results_scenario1_.05_1.6_for_uncertainty.csv',index_col=0)

In [17]:
rloss5, rmse5, tot_mse5 = compare_with_ground_truth(predict_choice_5, acs)

In [18]:
predict_choice_5.shape

(55798, 6)

In [19]:
predict_choice_1['rmse'] = rmse1
predict_choice_2['rmse'] = rmse2
predict_choice_3['rmse'] = rmse3
predict_choice_4['rmse'] = rmse4
predict_choice_5['rmse'] = rmse5

In [20]:
predict_choice_1.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5),P(mode6),rmse
3-4,0.0,0.0,0.0,3.877147,0.249570,1.873283,36787.252943
3-7,0.0,0.0,0.0,12.012387,1.458571,22.529042,36787.252943
3-9,0.0,0.0,0.0,0.140546,0.072559,1.786895,36787.252943
3-10,0.0,0.0,0.0,0.157064,0.124834,2.718103,36787.252943
3-11,0.0,0.0,0.0,3.853464,0.145975,0.000561,36787.252943


In [21]:
combined = predict_choice_1.append(predict_choice_2.append(predict_choice_3.append\
                                                           (predict_choice_4.append(predict_choice_5))))

In [22]:
# wm = lambda dfx: (dfx * np.exp(-dfx["rmse"])).sum() / np.exp(-dfx["rmse"]).sum()

In [532]:
combined.rmse.unique()

array([36787.25294336, 36394.52615727, 35921.7920947 , 36182.38317826,
       36583.11298344])

In [23]:
combined['rmse'] = combined.rmse.apply(lambda x: np.exp(1/x))

In [534]:
wm = lambda x: pd.Series(np.abs(np.average(x, weights= (x['rmse']),axis=0)))

In [24]:
combined.rmse.unique()/combined.rmse.unique().sum()

array([0.19999994, 0.2       , 0.20000007, 0.20000003, 0.19999997])

In [28]:
list(zip([(0.04,1.5),(0.05,1.5),(0.06,1.5),(0.5,1.4),(0.05,1.6)],combined.rmse.unique()/combined.rmse.unique().sum()))

[((0.04, 1.5), 0.19999993782503958),
 ((0.05, 1.5), 0.1999999964911128),
 ((0.06, 1.5), 0.20000006881019378),
 ((0.5, 1.4), 0.20000002871110628),
 ((0.05, 1.6), 0.19999996816254745)]

In [27]:
pd.DataFrame(([0.19999994, 0.2       , 0.20000007, 0.20000003, 0.19999997]))

,0
0,0.2
1,0.2
2,0.2
3,0.2
4,0.2


In [535]:
mean_combined = combined.groupby(combined.index).apply(wm)

In [536]:
mean_combined.columns = predict_choice_1.columns

In [537]:
mean_combined.drop(['rmse'],axis=1,inplace=True)

In [538]:
mean_combined.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5),P(mode6)
10-100,0.000000e+00,0.000000e+00,0.000000e+00,131.060201,4.939799,4.004003e-08
10-101,0.000000e+00,3.791167e-28,9.110685e-16,1.942899,18.874800,7.118230e+01
10-102,8.459467e-09,3.618705e-65,7.819331e-57,0.042130,5.670556,4.328731e+01
10-106,0.000000e+00,0.000000e+00,0.000000e+00,5.242844,2.022805,2.073435e+01
10-107,0.000000e+00,0.000000e+00,0.000000e+00,39.001455,1.200239,6.798306e+00


In [539]:
mean_combined.shape

(55798, 6)

In [540]:
mean_combined.dropna().shape

(55798, 6)

In [541]:
def weighted_std(values):
    average = (np.average(values, weights=(values['rmse']),axis=0))
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=(values['rmse']),axis=0)
    return (pd.Series(np.sqrt(np.abs(variance))))

In [542]:
std_combined = combined.groupby(combined.index).apply(weighted_std)

In [543]:
std_combined.columns = predict_choice_1.columns

In [544]:
std_combined.drop(['rmse'],axis=1,inplace=True)

In [546]:
std_combined.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5),P(mode6)
10-100,0.000000e+00,0.000000e+00,0.000000e+00,0.055687,0.055687,3.473214e-08
10-101,0.000000e+00,7.582332e-28,1.822137e-15,0.110773,0.089123,2.170443e-02
10-102,1.071153e-08,7.237408e-65,1.563866e-56,0.007989,0.025504,3.344761e-02
10-106,0.000000e+00,0.000000e+00,0.000000e+00,0.007416,0.004049,3.367155e-03
10-107,0.000000e+00,0.000000e+00,0.000000e+00,0.029121,0.000804,2.992347e-02


In [547]:
std_combined.shape

(55798, 6)

In [548]:
std_combined.dropna().shape

(55798, 6)

In [549]:
mean_combined.to_csv('Final_results/Uncertainty/mean_scenario_1.csv')

In [550]:
std_combined.to_csv('Final_results/Uncertainty/std_scenario_1.csv')

In [327]:
mean_combined.sum()

P(mode1)    5.606551e+04
P(mode2)    7.136258e+03
P(mode3)    2.463458e+03
P(mode4)    1.449029e+06
P(mode5)    2.282187e+05
P(mode6)    1.138438e+06
dtype: float64

In [399]:
std_combined.sum()

P(mode1)    3890.121024
P(mode2)    2322.527044
P(mode3)    4106.473318
P(mode4)    5561.334405
P(mode5)     237.574695
P(mode6)    4558.344969
dtype: float64